In [ ]:
from ultralytics import YOLO
import cv2
import os
import pandas as pd

def predict(chosen_model, img, classes=[], conf=0.5):
    if classes:
        results = chosen_model.predict(img, classes=classes, conf=conf)
    else:
        results = chosen_model.predict(img, conf=conf)

    return results


def predict_and_detect(chosen_model, img, classes=[], conf=0.5, rectangle_thickness=5, text_thickness=1):
    results = predict(chosen_model, img, classes, conf=conf)
    for result in results:
        for box in result.boxes:
            cv2.rectangle(img, (int(box.xyxy[0][0]), int(box.xyxy[0][1])),
                          (int(box.xyxy[0][2]), int(box.xyxy[0][3])), (255, 0, 255), rectangle_thickness)
            # cv2.putText(img, f"{result.names[int(box.cls[0])]}",
            #             (int(box.xyxy[0][0]), int(box.xyxy[0][1]) - 10),
            #             cv2.FONT_HERSHEY_PLAIN, 1, (255, 0, 0), text_thickness)
    return img, results



model = YOLO("best.pt")
path=os.getcwd()
for root, dirs, file in os.walk(path):
    for f in file:
        if ".jpeg" in f:
    # read the image
            image = cv2.imread(os.path.abspath(os.path.join(root,f)))
            result_img, results = predict_and_detect(model, image, classes=[], conf=0.5)
            
            with open(f"coord_{f}.txt",'a') as file:
                for result in results:
                    df=result.to_df()
                    df.to_csv(f"{f}_annots.csv")
                    # for box in result.boxes:
                    #      file.write(str(box)+"\n")
        # cv2.imshow("Image", result_img)
            cv2.imwrite(f"result{f}.png", result_img)
            cv2.waitKey(0)
            

In [ ]:
import os
import configparser
import cv2
import numpy as np
from pero_ocr.core.layout import PageLayout
from pero_ocr.document_ocr.page_parser import PageParser
import re
# from langchain_ollama import ChatOllama
# from langchain_core.messages.ai import AIMessage


In [ ]:


# Read config file.
config_path = "pero_eu_cz_print_newspapers_2022-09-26\\config.ini"
config = configparser.ConfigParser()
config.read(config_path)

# # Init the OCR pipeline. 
# # You have to specify config_path to be able to use relative paths
# # inside the config file.
page_parser = PageParser(config, config_path=os.path.dirname(config_path))


# Read the document page image.
path=os.getcwd()+"\\dossier_dexperience"
for f in os.listdir(path):
        
            
            input_image_path = f"dossier_dexperience\\{f}"
            image = cv2.imread(input_image_path, 1)

            # Init empty page content. 
            # This object will be updated by the ocr pipeline. id can be any string and it is used to identify the page.
            page_layout = PageLayout(id=input_image_path, page_size=(image.shape[0], image.shape[1]))

        # Process the image by the OCR pipeline
        # try:
            page_layout = page_parser.process_page(image, page_layout)


            page_layout.to_pagexml(f'dossier_output_experience\\output_page{f}.xml') # Save results as Page XML.
            page_layout.to_altoxml(f'dossier_output_experience\\output_ALTO{f}.xml') # Save results as ALTO XML.
            for region in page_layout.regions:
                
                liste_lignes = [f"Zone_texte",]

                for line in region.lines:
                    liste_lignes.append(line.transcription)

                with open(f'dossier_output_experience\\texte{f}.txt', 'a', encoding='utf-8') as txt:
                    txt.write('\n')
                    for element in liste_lignes:
                        txt.write(element +" ")
        # except:
        #     print(f"probleme avec {f}")
        #     pass

        # Render detected text regions and text lines into the image and
        # save it into a file.
            rendered_image = page_layout.render_to_image(image) 
            cv2.imwrite(f'results_full2\\{f}.png', rendered_image)

            # Save each cropped text line in a separate .jpg file.
            # for region in page_layout.regions:
            #     for line in region.lines:
            #         cv2.imwrite(f'file_id-{line.id}.jpg', line.crop.astype(np.uint8))
        

In [ ]:

def correction_mistral (texte):
    llm = ChatOllama(model="mistral", temperature=0 )
    messages = [("system","You are post correcting OCR results on a 1920 newspaper. If the text is empty or only composed of ponctuation return 'none'.",
    ),("human",f"{texte}")]
    ai_msg = llm.invoke(messages)
    return ai_msg
path=os.getcwd()+"\\resulttext_pre_cor"
for file in os.listdir(path):
  
    print(f"starting {file}")
    liste_corrigee =[]
    with open(f'resulttext_pre_cor\\{file}','r') as f:
        liste = (f.read()).split('Zone_texte')
    for line in liste : 
        # if re.match(r'\w', line):
        
            print(line)
            
            correc = correction_mistral(line)
            print(correc.pretty_repr())
            liste_corrigee.append(correc)
        # else:
        #       print(f"ça : {line} est pas bon")
        #       pass
    
    with open(f'resulttext_post_cor\\{file}2.txt', 'w') as txt:
        for element in liste_corrigee:
                        
                        txt.write(element.pretty_repr() + "\n")




In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import tqdm as notebook_tqdm

model = AutoModelForSeq2SeqLM.from_pretrained('pykale/bart-base-ocr')
tokenizer = AutoTokenizer.from_pretrained('pykale/bart-base-ocr')
generator = pipeline('text2text-generation', model=model.to('cuda'), tokenizer=tokenizer, device='cuda', max_length=1024)
path=os.getcwd()+"\\resulttext_pre_cor"
for file in os.listdir(path):
  
   
    liste_corrigee =[]
    with open(f'resulttext_pre_cor\\{file}','r') as f:
        liste = (f.read()).split('Zone_texte')
    for line in liste : 

        ocr = line
        pred = generator(ocr)[0]['generated_text']
        liste_corrigee.append(pred)
        print(line)
        print(pred)
    with open(f'resulttext_post_cor\\{file}2.txt', 'w') as txt:
        for element in liste_corrigee:
                        
                        txt.write(element + "\n")
